# RAG for private documents

How can LLMs learn new knowledge? 

    - Fine tuning on a training set
    - MOdel inputs

The recommended approach is to use model with embedded-based search.

1. Prepare the search data
    - load data into langchain documents
    - split the document into chunks
    - embed the chunks into numeric vectors
    - save the chunks and embeddings to a vector database
      
2. Search, once per query
    - Embed a user's question
    - Using the question embedding and the chunk embeddings, rank the vectors by similarity to the question embedding, whee the neares vector represents chunks most relevant.

3. Ask
    - Insert the question and the most relevant chunks into a message to a GPT model
    - Return GPT answer.



# Load the documents

In [1]:
# !pip install -q pypdf  #already installed so commenting out 

In [2]:
# !pip install -q docx2txt   #already installed so commenting out 

In [3]:
# !pip install wikipedia -q   # alrady loaded so commenting out

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)


###############  loading PDF, DOCX and TXT files as LangChain Documents
def load_document(file):
    import os
    name, extension = os.path.splitext(file)

    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading {file}')
        loader = PyPDFLoader(file) # cam also be used as pointer to online pdf in arg
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    # elif extension == '.txt':
    #     from langchain.document_loaders import TextLoader
    #     loader = TextLoader(file)
    else:
        print('Document format is not supported!')
        return None

    data = loader.load()
    return data


##############  Loading the data from wikipedia
def load_from_wikipedia(query, lang="en",load_max_docs=2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang, load_max_docs=load_max_docs)
    data = loader.load()
    return data

############# Data chunking before embedding int vector database
def chunk_data(data, chunk_size=256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter # recommended for langchain : tries to split \\n \n and whitespace
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

############ Calculate the cost of embedding
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-3-small')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.00002:.6f}')
    
    

# Embedding and Uploading a Vector Database to Pinecone

In [5]:
def insert_or_fetch_embeddings(index_name, chunks):
    # importing the necessary libraries and initializing the Pinecone client
    import pinecone
    from langchain_community.vectorstores import Pinecone
    from langchain_openai import OpenAIEmbeddings
    from pinecone import PodSpec

    
    pc = pinecone.Pinecone()
        
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  # 512 works as well

    # loading from existing index
    if index_name in pc.list_indexes():
        print(f'Index {index_name} already exists. Loading embeddings ... ', end='')
        vector_store = Pinecone.from_existing_index(index_name, embeddings)
        print('Ok')
    else:
        # creating the index and embedding the chunks into the index 
        print(f'Creating index {index_name} and embeddings ...', end='')

        # creating a new index
        pc.create_index(
            name=index_name,
            dimension=1536,
            metric='cosine',
            spec=PodSpec(
                environment='gcp-starter'
            )
        )

        # processing the input documents, generating embeddings using the provided `OpenAIEmbeddings` instance,
        # inserting the embeddings into the index and returning a new Pinecone vector store object. 
        vector_store = Pinecone.from_documents(chunks, embeddings, index_name=index_name)
        print('Ok')
        
    return vector_store

# Delete a pinecone index

In [6]:
def delete_pinecone_index(index_name='all'):
    import pinecone
    pc = pinecone.Pinecone()
    
    if index_name == 'all':
        indexes = pc.list_indexes().names()
        print('Deleting all indexes ... ')
        for index in indexes:
            pc.delete_index(index)
        print('Ok')
    else:
        print(f'Deleting index {index_name} ...', end='')
        pc.delete_index(index_name)
        print('Ok')

# Running the code

In [7]:
# Loading the pdf and docx document into LangChain 

data = load_document('files/us_constitution.pdf')   # for PDF
# data = load_document('files/the_great_gatsby.docx')   # for docx
print(data[0].page_content)
print(data[10].metadata)
print(f'You have {len(data)} pages in your document')
print(f'There are  {len(data[20].page_content)} characters in page')


Loading files/us_constitution.pdf
The
United
States
Constitution
W e
the
People
of
the
United
States,
in
Order
to
form
a
more
perfect
Union,
establish
Justice,
insure
domestic
T ranquility ,
provide
for
the
common
defence,
promote
the
general
W elfare,
and
secure
the
Blessings
of
Liberty
to
ourselves
and
our
Posterity ,
do
ordain
and
establish
this
Constitution
for
the
United
States
of
America.
The
Constitutional
Con v ention
Article
I
Section
1:
Congress
All
legislative
Powers
herein
granted
shall
be
vested
in
a
Congress
of
the
United
States,
which
shall
consist
of
a
Senate
and
House
of
Representatives.
Section
2:
The
House
of
Representatives
{'source': 'files/us_constitution.pdf', 'page': 10}
You have 41 pages in your document
There are  1137 characters in page


In [8]:
# Loading the data from Wikipedia

data = load_from_wikipedia('GPT-4', lang="pt")
print(data[0].page_content)

C:\Users\Vagner Machado\AppData\Roaming\Python\Python312\site-packages\langchain_community\document_loaders\wikipedia.py:6: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_community.utilities.wikipedia import WikipediaAPIWrapper


Generative Pre-trained Transformer 4 (GPT-4) é um modelo de linguagem grande multimodal criado pela OpenAI e o quarto modelo da série GPT.  Foi lançado em 14 de março de 2023, e se tornou publicamente aberto de forma limitada por meio do ChatGPT Plus, com o seu acesso à API comercial sendo provido por uma lista de espera. Sendo um transformador, foi pré-treinado para prever o próximo token (usando dados públicos e "licenciados de provedores terceirizados"), e então foi aperfeiçoado através de uma técnica de aprendizagem por reforço com humanos. 
A empresa Microsoft, após o lançamento do modelo, confirmou que versões do Bing utilizando o GPT estavam, de fato, utilizando o modelo mais recente da OpenAI antes de seu lançamento oficial. 


== Capacidades ==
Diferentemente de seu predecessor, o GPT-3, o GPT-4 é capaz de processar imagens como entrada, não apenas texto, analisando o conteúdo da imagem de forma semelhante a um humano, e emitindo uma saída em forma de texto.
Pesquisadores da M

In [9]:
# # Splitting the document into chunks
chunks = chunk_data(data, chunk_size=256)
print(len(chunks))
print(chunks[10].page_content)
print_embedding_cost(chunks)
# # Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
# vector_store = create_embeddings_chroma(chunks)

37
inteligência artificial mais poderosos que o GPT-4. Entre os signatários estão Elon Musk, Steve Wozniak e Emad Mostaque, além de pesquisadores da DeepMind.
Total Tokens: 1849
Embedding Cost in USD: 0.000037


In [10]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [11]:
index_name = "askadoc"
vector_store = insert_or_fetch_embeddings(index_name, chunks)

Creating index askadoc and embeddings ...Ok


# Getting into QA with similarity search

In [12]:
def ask_and_get_answer(vector_store, q, k=3):
    from langchain.chains import RetrievalQA
    from langchain_openai import ChatOpenAI

    llm = ChatOpenAI(model='gpt-3.5-turbo', temperature=1)
    retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': k})
    chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)
    answer = chain.invoke(q)
    return answer

In [13]:
q = "What is the whole document about?"
answer = ask_and_get_answer(vector_store, q)
print(answer["result"])

I'm sorry, but I don't have access to the entire document you are referring to. If you could provide more specific information or context, I may be able to help you better.


In [15]:
import time
i = 1
print("Write QUIT or EXIT to leave program")
while True:
    q = input(f"Question #{i}: ")
    i = i + 1
    if q.lower() in ["quit","exit"]:
         print("Leaving the program.")
         time.sleep(2)
         break;
    answer = ask_and_get_answer(vector_store, q)
    print(f"\nAnswer: " + answer["result"] + "\n\n" )
    print("-----" * 25)
    

Write QUIT or EXIT to leave program
Leaving the program.


In [ ]:
delete_pinecone_index()

Deleting all indexes ... 
Ok


In [ ]:
data = load_from_wikipedia("ChatGPT", "pt") #pt for portuguese language 
chunks = chunk_data(data)
index_name = "chatgpt"
vector_store = insert_or_fetch_embeddings(index_name, chunks)

Creating index chatgpt and embeddings ...Ok


In [ ]:
q = "O que e' ChatGPT?"
answer = ask_and_get_answer(vector_store, q)
print(answer["result"])

ChatGPT é um modelo de linguagem desenvolvido pela OpenAI que utiliza inteligência artificial para responder perguntas, manter conversas e fornecer informações em linguagem natural. Ele é projetado para gerar respostas coerentes e relevantes com base no contexto da conversa.


# Chroma

A good alternative to Pinecone. This is in memory and does not need to deal with create and delete indexes.

### NOTE that due to dependencies I was not able to install Chroma. 

In [ ]:
!pip install -q chromadb

In [1]:
def create_embeddings_chroma(chunks, persist_directory="./chroma_db"):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate an embedding model from OpenAI (smaller version for efficiency)
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536)  

    # Create a Chroma vector store using the provided text chunks and embedding model, 
    # configuring it to save data to the specified directory 
    vector_store = Chroma.from_documents(chunks, embeddings, persist_directory=persist_directory) 

    return vector_store  # Return the created vector store

def load_embeddings_chroma(persist_directory='./chroma_db'):
    from langchain.vectorstores import Chroma
    from langchain_openai import OpenAIEmbeddings

    # Instantiate the same embedding model used during creation
    embeddings = OpenAIEmbeddings(model='text-embedding-3-small', dimensions=1536) 

    # Load a Chroma vector store from the specified directory, using the provided embedding function
    vector_store = Chroma(persist_directory=persist_directory, embedding_function=embeddings) 

    return vector_store  # Return the loaded vector store

In [ ]:
# Loading the pdf document into LangChain 
data = load_document('files/rag_powered_by_google_search.pdf')

# Splitting the document into chunks
chunks = chunk_data(data, chunk_size=256)

# Creating a Chroma vector store using the provided text chunks and embedding model (default is text-embedding-3-small)
vector_store = create_embeddings_chroma(chunks)

In [ ]:
# Asking questions
q = 'What is Vertex AI Search?'
answer = ask_and_get_answer(vector_store, q)
print(answer)
print(answer['result'])

In [ ]:
# We can't ask follow-up questions. There is no memory (chat history) available.
q = 'Multiply that number by 2.'
answer = ask_and_get_answer(vector_store, q)
print(answer['result'])

# Adding Memory / Chat History

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain  # Import class for building conversational AI chains 
from langchain.memory import ConversationBufferMemory  # Import memory for storing conversation history

In [ ]:
# Instantiate a ChatGPT LLM (temperature controls randomness)
llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0) 

# Configure vector store to act as a retriever (finding similar items, returning top 5)
retriever = vector_store.as_retriever(search_type='similarity', search_kwargs={'k': 5})  


# Create a memory buffer to track the conversation
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

crc = ConversationalRetrievalChain.from_llm(
    llm=llm,  # Link the ChatGPT LLM
    retriever=retriever,  # Link the vector store based retriever
    memory=memory,  # Link the conversation memory
    chain_type='stuff',  # Specify the chain type
    verbose=False  # Set to True to enable verbose logging for debugging
)

In [ ]:
# create a function to ask questions
def ask_question(q, chain):
    result = chain.invoke({'question': q})
    return result

In [ ]:
data = load_document('files/rag_powered_by_google_search.pdf')
chunks = chunk_data(data, chunk_size=256)
vector_store = create_embeddings_chroma(chunks)